# Neural Networks

To get started, please install `keras`. You can do so using:

```
conda install keras
```

The amazing Keras documentation is available here: https://keras.io. They provide _tons_ of great, real-world examples. It was created by Francois Chollet. He's great to follow on Twitter.

Keras does a few really nice things for us:

* Keras lets us write models using very simple syntax. It provides the lego building blocks for you to make almost _any_ kind of neural network you can imagine.
* Keras "wraps" tensorflow. Tensorflow is a library produced by Google for doing very efficient linear algebra and optimization. Fortunately, neural networks are all about linear algebra and optimization.
* Tensorflow is really hard to just jump in to. Keras hides the messy parts of Tensorflow from you but allows you to use them when you need to. Therefore it is great for starting projects easily, even if those projects eventually become very complicated and require custom code.
* Neural networks can be trained very efficiently on certain graphics processing units (GPUs). Writing code for GPUs is very hard. Keras will determine whether your GPU (if you have one) can run your code and will then allow you to use either your GPU or your CPU without making any code changes. 

## Getting Started

Let's import the usual suspects: `pandas`, `matplotlib`, `numpy`, `seaborn`, and `sklearn`. 


In [ ]:
## Data frames
import pandas as pd

## Matrices and algebra
import numpy as np

## Plotting
import matplotlib.pyplot as plt

## Heatmaps
from seaborn import heatmap

## Confusion matrices
from sklearn.metrics import confusion_matrix


## Importing Keras

Keras is a library that allows us to build very powerful neural networks of all types using a lego style building block approach. We make new neural networks by defining an input, an ouput, and the layers inbetween.

Keras comes with two APIs: 

1. The Sequential API is for the most common types of neural networks; those with one input and one output. We will use this today.
2. Keras also offers the Model API that provides more flexibility for neural networks with complicated inputs and outputs.

To get started, let's build our own linear regression in Keras using the Sequential API. 

## Example 1: Linear Regression

Let's estimate a model for which we know the population model. The true model that generated the data (below) is a linear regression of the form $y = 2.5 + 3x + u$. 

In [ ]:
## Make an X,y training set of size 100
X_train = np.random.uniform(0,10,100)
y_train = 2.5 + 3*X_train + np.random.normal(0,1,100)

## Make an X,y test set of size 50
X_test = np.random.uniform(10,20,50)
y_test = 2.5 + 3*X_test + np.random.normal(0,1,50)

## Plot the training set
plt.scatter(X_train,y_train)

Cool. Now we want to fit a shallow neural network to these data. For each sample, the network will input one value (x) and output one value (y). Keras will insert a bias term (intercept) by default. We won't have to worry about that. First, let's define the network.

If you'd like to see the documentation on the Sequential API in Keras, go here: https://keras.io/getting-started/sequential-model-guide/

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

## Here, we define a new Sequential model.
shallow_linear_model = Sequential()

## Now, let's add a single layer (the output).
shallow_linear_model.add( Dense(units=1, input_shape=(1,)) )

## Next, we have to compile our model.
## Compiling our model tells Keras what optimizer
## we want to use and what metrics we should report.

## Here, we are optimizing our parameters using
## sgd or Stochastic Gradient Descent.
## We are minimizing the mean squared error, just
## like in OLS linear regression.
shallow_linear_model.compile(loss="mean_squared_error",
                             optimizer="sgd")

## Now, we fit our model to the data. This similar
## to how we fit models in scikit-learn.
shallow_linear_model.fit(x=X_train,
                         y=y_train,
                         epochs=100,
                         batch_size=10)

Great! We've fit our first neural network. Let's see what predictions it makes for our training set.

In [ ]:
## Plot the training set
plt.scatter(X_train,y_train)
## Plot the predicted values for the training set
plt.scatter(X_train, shallow_linear_model.predict(X_train))

So, how do we do on the test set?

In [ ]:
## Plot the training set
plt.scatter(X_test,y_test)
## Plot the predicted values for the training set
plt.scatter(X_test, shallow_linear_model.predict(X_test))

The out-of-sample fit may be a little bit off. This is because neural networks are optimized iteratively. The parameters update a little bit every epoch and so it may take dozens or hundreds of epochs for the network to converge to the best possible fit.

We can use the `.get_weights()` method to recover the parameters of our model. We only have two weights: the intercept and the weight on the single 1-unit (neuron) Dense layer. These should match our regression function $y=2.5 + 3*X$.

In [ ]:
shallow_linear_model.get_weights()

## Example 2: Logistic Regression

Linear regression is boring. Let's do a logistic regression. Remember that we use logistic regression when we have a binary outcome variable. First thing to do is generate some nice looking data.

In [ ]:
## We'll skip the training and testing data this time.
## Just to make things easy.
X = np.random.uniform(0,20,100)
y = (X + np.random.normal(0,1,100) > 10).astype("int")

## Plot the training set
plt.scatter(X,y)

Now let's define a new Sequential model called `shallow_logistic_model`. This will look just like our linear model (one input X and one output y). However, as we know from regression courses, logistic regression has a link function - a non-linear function that takes our real values and outputs values between 0 and 1. Logistic regression uses a logit link function. Another name for this is a "sigmoid" function (below). A link function in neural networks is referred to as an activation function. A single model may have many (millions) of activation functions. Typically we have one activation function per neuron.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1920px-Logistic-curve.svg.png"></img>

This code is going to take longer to converge. In my experiments, the SGD optimizer wants 2000-3000 epochs to converge to a good fit. There are other optimizers that will be much faster. You can find documentation about them here: https://keras.io/optimizers/

In [ ]:
## Initialize our model.
shallow_logistic_model = Sequential()

## Add a single layer with 1 input, 1 output, and a sigmoid activation.
shallow_logistic_model.add( Dense(units=1, input_shape=(1,), activation="sigmoid") )

## Compile our model.
## This time we'll use binary_crossentropy.
## This is also called log loss.
shallow_logistic_model.compile(loss="binary_crossentropy",
                               optimizer="sgd")

## Fit the model to our data!
shallow_logistic_model.fit(x=X,
                           y=y,
                           epochs=3000,
                           batch_size=50)

Below, we can plot the predicted values from our model. Remember they will be probabilities and so they will be between 0 and 1.

In [ ]:
## Plot the training set
plt.scatter(X,y)
## Plot the predicted values for the training set
plt.scatter(X, shallow_logistic_model.predict(X))
# ## Plot the rounded predicted values
# plt.scatter(X, np.round(shallow_logistic_model.predict(X))+0.1)

## Example 3: Multiple Regression

So far we've made two shallow bivariate regression models using neural networks. But, that's pretty damn cool. You just built a bivariate logistic regression model from scratch (almost)! Let's make a multiple logistic regression model. In this case we'll have one output and many inputs. It will still be shallow - no hidden layers.

In [ ]:
## Make some random data
x1 = np.random.normal(0,1,100)
x2 = np.random.uniform(0,10,100)
x3 = np.random.poisson(7,100)

## Compute a Y vector as a function of the x's
y = (0.5 + 3*x1 - 2*x2 + 0.5*x3 + np.random.normal(0,1,100)) > 0

## What proportion of Y is positive (1) values?
print(np.mean(y))


We need our x variables to be in a single X matrix where each variable is a column. We can use `np.column_stack` to stack these arrays as columns in a single matrix.

In [ ]:
X = np.column_stack([x1,x2,x3])
print(X.shape)

### Exercise 1

Let's initialize a new `model` of type `Sequential`. In fact, this model will look a lot like our last model, but the `input_shape` needs to be a tuple with the number of columns in our X matrix that should look like: `input_shape=(3,)`. When you fit the model, try 1000 epochs and a batch size of 64. Also, in the `.fit` method, add `class_weight="auto"`.

In [ ]:
## Initialize a new model called model


## Add a dense layer with sigmoid activation


## Compile the model with binary_crossentropy loss and sgd optimizer


## Fit the model to our data X and y



Let's compute a confusion matrix for our model.

In [ ]:
## First compute a normalized confusion matrix
cnf_mat = confusion_matrix(y, np.round(model.predict(X)), normalize="true")

## Then plot the matrix as a heatmap
## Rows: True values
## Columns: Predicted values
heatmap(cnf_mat, square=True, annot=True, cbar=False, cmap="YlGnBu")


## Example 4: Multiclass Classification

Neural networks are very flexible. So far, our output has been a single value (y) for every row of data. Now, let's output a vector for every row of data. In particular, we are going to classify the iris data (again). Remember that there are three kinds of iris:

* versicolor
* setosa
* virginica

Let's represent them as a "one-hot-encoded" vector, like so:

```
[1,0,0] = versicolor
[0,1,0] = virginica
[0,0,1] = setosa
```

In [ ]:
iris = pd.read_csv("iris.csv")

## Let's make a new matrix that is n by 3
## It is all zeros to start
y_iris = np.zeros((iris.shape[0], 3))

## Now, fill in column 0 with 1 when iris["Species"] is "versicolor"
y_iris[:,0] = iris["Species"]=="versicolor"
## Fill in column 1 with 1 when iris["Species"] is "virginica"
y_iris[:,1] = iris["Species"]=="virginica"
## Fill in column 2 with 1 when iris["Species"] is "setosa"
y_iris[:,2] = iris["Species"]=="setosa"


We just made our dependent variable (output). If you print it, you will see that it is actually $(n \times 3)$. We will print the first 10 rows below.

In [ ]:
print(y_iris.shape)
y_iris[0:10]

Now let's make our x_iris dataframe. We can just subset the fields (columns) we're going to use to predict species. To make this hard, we're only going to use Sepal Length and Sepal Width.

In [ ]:
x_iris = iris[["SepalLength","PetalWidth"]]

Great! Now we're ready to build a model. Let's start with a simple linear model to begin with. This is going to work... poorly. First, let's talk about all of the pieces:

* `Dense(3...` Means that the dense layer will have 3 output values. One per flower species.
* `input_shape=(4,)` Means that the input layer has 4 features (Petal/Sepal Length/Width).
* `activation="softmax"` Softmax is an activation function for multiclass problems. It takes the output of size $1 \times 3$ and normalizes all three values so they add up to 1. That way, each value is a class predicted probability.
* `loss="categorical_crossentropy"` Categorical crossentropy is a generalization of logarithmic loss for multiple classes. It penalizes models that make bad predictions.

In [ ]:
## Initialize our model
model = Sequential()

## Add a single densly-connected layer
model.add( Dense(3, input_shape=(2,), activation="softmax") )

## Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

## Fit the model to our data
model.fit(x_iris,
          y_iris,
          epochs=1000,
          batch_size=25)

Just for fun, let's print a summary of our model. This describes the layers and the number of parameters we are estimating.

In [ ]:
model.summary()

Okay, let's first make some class predictions for our training data. We can do this more-or-less the same way that we would above.

In [ ]:
iris_predictions = model.predict(x_iris)
print(iris_predictions[0:10])

Actually, these are probabilities. That makes them a little hard to work with. Let's figure out the guesses our model made for each flower by finding out which columns is the maximum for every row. We can use `np.argmax` to do this.

In [ ]:
iris_predictions = np.argmax(iris_predictions, axis=1)
print(iris_predictions[0:10])

Great! Now let's plot the training set predictions and our model predictions.

In [ ]:
## First the true versicolor
plt.scatter(iris.loc[iris.Species=="versicolor","SepalLength"], 
            iris.loc[iris.Species=="versicolor","PetalWidth"])
## Then the true virginica
plt.scatter(iris.loc[iris.Species=="virginica","SepalLength"], 
            iris.loc[iris.Species=="virginica","PetalWidth"])
## Then the true setosa
plt.scatter(iris.loc[iris.Species=="setosa","SepalLength"], 
            iris.loc[iris.Species=="setosa","PetalWidth"])

In [ ]:
## First the predicted versicolor (column 0)
plt.scatter(iris.loc[iris_predictions==0,"SepalLength"], 
            iris.loc[iris_predictions==0,"PetalWidth"])
## Then the predicted virginica (column 1)
plt.scatter(iris.loc[iris_predictions==1,"SepalLength"], 
            iris.loc[iris_predictions==1,"PetalWidth"])
## Finally the predicted setosa (column 2)
plt.scatter(iris.loc[iris_predictions==2,"SepalLength"], 
            iris.loc[iris_predictions==2,"PetalWidth"])

## Example 5: Deep Learning for Flowers!

That model we just made is still a shallow model. It has an input layer with 2 features (sepal length and sepal width) and and output layer with 3 categories. But there are no hidden layers. You can see this in the predictions that it makes. The predictions are all just lines that divide the data into three sections. But, we could make these a little curvier to fit the data better. Let's try that. In this problem, you'll make a deep network with some non-linear hidden layers to see if we can do any better than our shallow network!

### Exercise 2

I have copied the shallow model below. Add one or more hidden layers with as many units (neurons) as you like and `activation="tanh"`. `tanh` is the hyperbolic tangent function. It is a non-linear function that will allow our model to learn some curves.

In [ ]:
## Initialize our model
model = Sequential()

## Add one or more Dense layers here
## Use tanh for the activation
## Use as many neurons as you like



## Add a single densly-connected layer for output
model.add( Dense(3, activation="softmax") )

## Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

## Fit the model to our data
model.fit(x_iris,
          y_iris,
          epochs=1000,
          batch_size=25)

Below is the code to make predictions with your model and plot the result.

In [ ]:
iris_predictions = model.predict(x_iris)
iris_predictions = np.argmax(iris_predictions, axis=1)

## First the predicted versicolor (column 0)
plt.scatter(iris.loc[iris_predictions==0,"SepalLength"], 
            iris.loc[iris_predictions==0,"PetalWidth"])
## Then the predicted virginica (column 1)
plt.scatter(iris.loc[iris_predictions==1,"SepalLength"], 
            iris.loc[iris_predictions==1,"PetalWidth"])
## Finally the predicted setosa (column 2)
plt.scatter(iris.loc[iris_predictions==2,"SepalLength"], 
            iris.loc[iris_predictions==2,"PetalWidth"])

So, how many parameters does your model have? A parameter is like a coefficient from linear regression. So a linear model with three independent variables and an intercept has four parameters. We can use `model.summary()` to print a summary of every layer of your model that tells you how many parameters you have.

In [ ]:
model.summary()

## Example 6: Enough Messing Around. Let's Do mnist!

Alright. So neural networks excel at problems with unusual input types and correlation structures. For example: images! Let's do some hand written digit classification using mnist.

In [ ]:
import mnist

## First, we load the MNIST training and testing data
images_train = mnist.train_images()
images_train_labels = mnist.train_labels()
images_test = mnist.test_images()
images_test_labels = mnist.test_labels()

Mnist data are images. Each image is a hand-written digit. Convolutional neural networks are particularly great for images. You can think of a convolutional layer as a "blur" or "sharpen" that gets applied to each image. A single convolutional layer may have dozens or hundreds of these filters and each filter will have a particular shape. Some may sharpen horizontal lines and others vertical lines, for example. Successive layers of this convolutional network combine these filters to make complex shapes that represent, in our case, classes of numbers. 

First, though, we have to convert our image labels (0-9) to one-hot-encoded vectors like so:

* 0: [1,0,0,0,0,0,0,0,0,0]
* 1: [0,1,0,0,0,0,0,0,0,0]
* 2: [0,0,1,0,0,0,0,0,0,0]
* ...
* 9: [0,0,0,0,0,0,0,0,0,1]

In [ ]:
images_train_onehot = np.zeros((images_train_labels.size, 10))
images_train_onehot[np.arange(images_train_labels.size), images_train_labels] = 1

print(images_train_onehot[0:10])

Because Keras accepts images in either 1 color (grayscale) or 3 colors (red,green,blue), our image array should be $N \times x \times y \times 1$ for grayscale. X and Y are the length and width of the images. However, that one on the end seems redundant. In our dataset, the one is omitted. You can see this below.

In [ ]:
images_train.shape

But, Keras really needs that 1 to be there or it'll get mad. So let's add a dimension. It doesn't actually change the data.

In [ ]:
images_train = np.expand_dims(images_train, axis=3)
images_test = np.expand_dims(images_test, axis=3)
images_train.shape

Below, we define a deep neural network with two hidden convolutional layers. Our convolutions are 2 dimensional (length and width) and they have the following properties:

* `filters=12` We have 12 unique filters for each layer. Think of a filter as a specific blur or sharpen function.
* `kernel_size=(3,3)` The size of our each filter is 3 pixels by 3 pixels. In other words, they're very small filters. Each filter is passed over the image so the same 3x3 filter gets applied to all parts of the input image.
* `activation="tanh"` We're using `tanh` again because it is nonlinear and will allow our model to pick up on more complext relationships between pixels in our images. Also, it will keep our weights from getting too big.

We add a flatten layer after our convolutional layers. Because the convolutional layers still output images (that have been filtered through our filters), we need some way to turn those images into a single vector of features. Flatten just takes each image and unrolls it into a long 1D array.

Finally, we add a Dense layer on the end with 10 output neurons and softmax activation. This is just a multiclass classifier like before.

We compile the model the same as before and fit it just like before. We only use one epoch this time because we have 60,000 training examples. That should be enough examples for the model to fit nicely in one epoch. If we gave it more epochs (like 10), the model would probably perform a little better. Eventually you will get diminishing returns.

In [ ]:
from keras.layers import Conv2D, Flatten

mnist_model = Sequential()

mnist_model.add( Conv2D(filters=12, kernel_size=(3,3), input_shape=(28,28,1), activation="tanh") )
mnist_model.add( Conv2D(filters=12, kernel_size=(3,3), activation="tanh") )
mnist_model.add( Flatten() )
mnist_model.add( Dense(10, activation="softmax") )

mnist_model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

mnist_model.fit(images_train,
          images_train_onehot,
          batch_size=32,
          epochs=1)

Just like before, let's compute class predictions for our training set.

In [ ]:
## Let's compute predictions for our training data
mnist_predictions = mnist_model.predict(images_train, batch_size=32)
mnist_predictions = np.argmax(mnist_predictions, axis=1)

Now, plot the confusion matrix of our predictions versus the true values.

In [ ]:
## Now compute a normalized confusion matrix
cnf_mat = confusion_matrix(images_train_labels, mnist_predictions, normalize="true")

## Then plot the matrix as a heatmap
## Rows: True values
## Columns: Predicted values
plt.subplots(figsize=(6,6))
heatmap(cnf_mat, square=True, annot=True, cbar=False, cmap="YlGnBu")

Finally, let's compute predictions on our test data. How well are we doing here?

In [ ]:
mnist_test_predictions = mnist_model.predict(images_test, batch_size=32)
mnist_test_predictions = np.argmax(mnist_test_predictions, axis=1)

In [ ]:
## Now compute a normalized confusion matrix
cnf_mat = confusion_matrix(images_test_labels, mnist_test_predictions, normalize="true")

## Then plot the matrix as a heatmap
## Rows: True values
## Columns: Predicted values
plt.subplots(figsize=(6,6))
heatmap(cnf_mat, square=True, annot=True, cbar=False, cmap="YlGnBu")